<a href="https://colab.research.google.com/github/saakolch/PYAS/blob/main/tokenizer_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers sentencepiece

In [2]:
from datasets import load_dataset

raw_dataset = load_dataset('code_search_net', 'python')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [3]:
raw_dataset['train']

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [6]:
raw_dataset['train'][123]['whole_func_string']

'def slicing(args, length):\n    """Internally used."""\n    if isinstance(args, tuple):\n        for arg in args:\n            yield from slicing_singlevalue(arg, length)\n    else:\n        yield from slicing_singlevalue(args, length)'

In [21]:
# Don't uncomment the following line unless your dataset is small!
def get_training():
    dataset = raw_dataset['train']
    for idx in range(0, len(dataset), 1000):
        samples = dataset[idx: idx + 1000]
        yield samples['whole_func_string']

training_corpus = get_training()

In [12]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Make up the vocabulary after a bit examples:

In [13]:
example = '''def: add_numbers(a,b):
    """Add the two numbers 'a' and 'b'."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 ':',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'b',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 "Ġ'",
 'a',
 "'",
 'Ġand',
 "Ġ'",
 'b',
 '\'."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

Ġ and Ċ denote spaces and newlines respectively. So we are going to train new, suitable tokenizer for find more specific one for computer language

In [22]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [23]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 ':',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'b',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 "Ġ'",
 'a',
 "'",
 'Ġand',
 "Ġ'",
 'b',
 '\'."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

This is quite more compact representation compared to plain English tokenizer:

In [24]:
len(tokens), len(old_tokenizer.tokenize(example))

(28, 36)

One more example

In [25]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'randn',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

In order to use it later we have to save the new tokenizer

In [26]:
tokenizer.save_pretrained('code-search-net-tokenizer')

('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

So now we can use a folder named code-search-net-tokenizer which contains all the files that the tokenizer needs to reload. We can upload it to the Hugging Face Hub by logging into account

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [28]:
tokenizer.push_to_hub("code-search-net-tokenizer")

CommitInfo(commit_url='https://huggingface.co/forwins/code-search-net-tokenizer/commit/6a0c4d20a6551250648e343a4260e27b2369e905', commit_message='Upload tokenizer', commit_description='', oid='6a0c4d20a6551250648e343a4260e27b2369e905', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
# Replace 'forwins' below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("forwins/code-search-net-tokenizer")

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/822k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/467k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]